In [68]:
import os
import nltk
import spacy
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, chi2

In [69]:
# NLTK setup
NLTK_PATH = "/home/codespace/nltk_data"
os.makedirs(NLTK_PATH, exist_ok=True)
nltk.download("stopwords", download_dir=NLTK_PATH)
nltk.data.path.append(NLTK_PATH)
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
import subprocess
try:
    nlp = spacy.load("en_core_web_sm")
except:
    subprocess.run(["python3", "-m", "spacy", "download", "en_core_web_sm"])
    nlp = spacy.load("en_core_web_sm")


In [71]:
# Exercise 1: Preprocessing
reviews = [
    "I absolutely loved this movie, it was fantastic!",
    "The film was boring and too long.",
    "What an incredible performance by the lead actor.",
    "Terrible plot and poor acting.",
    "A fun and exciting experience from start to finish!"
]
labels = [1, 0, 1, 0, 1]

In [72]:
def nltk_preprocess(text):
    # Fallback tokenizer (no punkt needed)
    tokens = [word for word in text.lower().split() if word.isalpha()]
    return [word for word in tokens if word not in stop_words]

def spacy_preprocess(text):
    doc = nlp(text.lower())
    return [token.text for token in doc if token.is_alpha and not token.is_stop]


In [73]:
print("=== Preprocessing Comparison ===")
for i, review in enumerate(reviews):
    print(f"\nReview {i+1}: {review}")
    nltk_tokens = nltk_preprocess(review)
    spacy_tokens = spacy_preprocess(review)
    print(f"NLTK Tokens: {nltk_tokens} | Count: {len(nltk_tokens)}")
    print(f"spaCy Tokens: {spacy_tokens} | Count: {len(spacy_tokens)}")

=== Preprocessing Comparison ===

Review 1: I absolutely loved this movie, it was fantastic!
NLTK Tokens: ['absolutely', 'loved'] | Count: 2
spaCy Tokens: ['absolutely', 'loved', 'movie', 'fantastic'] | Count: 4

Review 2: The film was boring and too long.
NLTK Tokens: ['film', 'boring'] | Count: 2
spaCy Tokens: ['film', 'boring', 'long'] | Count: 3

Review 3: What an incredible performance by the lead actor.
NLTK Tokens: ['incredible', 'performance', 'lead'] | Count: 3
spaCy Tokens: ['incredible', 'performance', 'lead', 'actor'] | Count: 4

Review 4: Terrible plot and poor acting.
NLTK Tokens: ['terrible', 'plot', 'poor'] | Count: 3
spaCy Tokens: ['terrible', 'plot', 'poor', 'acting'] | Count: 4

Review 5: A fun and exciting experience from start to finish!
NLTK Tokens: ['fun', 'exciting', 'experience', 'start'] | Count: 4
spaCy Tokens: ['fun', 'exciting', 'experience', 'start', 'finish'] | Count: 5


### NLTK vs spaCy Tokenization – Analysis

- **Token Count**: NLTK and spaCy produced slightly different token counts due to differences in tokenizer rules.
- **Stopword Removal**: Both libraries effectively removed common English stopwords like "the", "was", etc.
- **spaCy Advantage**: spaCy handled punctuation and named entities more robustly, which is beneficial in NLP tasks involving context or named entity recognition.
- **NLTK Simplicity**: NLTK's pipeline is lightweight and works well for basic tasks or when resources are limited.

Overall, spaCy gives more linguistically accurate tokenization, but NLTK is more flexible for custom pipelines.


In [74]:
# Exercise 2: Feature Extraction

print("\n=== CountVectorizer vs TF-IDF ===")
count_vec = CountVectorizer()
tfidf_vec = TfidfVectorizer()

X_count = count_vec.fit_transform(reviews)
X_tfidf = tfidf_vec.fit_transform(reviews)

print("CountVectorizer Features:", count_vec.get_feature_names_out())
print("TF-IDF Features:", tfidf_vec.get_feature_names_out())

print("\n=== N-gram Analysis ===")


=== CountVectorizer vs TF-IDF ===
CountVectorizer Features: ['absolutely' 'acting' 'actor' 'an' 'and' 'boring' 'by' 'exciting'
 'experience' 'fantastic' 'film' 'finish' 'from' 'fun' 'incredible' 'it'
 'lead' 'long' 'loved' 'movie' 'performance' 'plot' 'poor' 'start'
 'terrible' 'the' 'this' 'to' 'too' 'was' 'what']
TF-IDF Features: ['absolutely' 'acting' 'actor' 'an' 'and' 'boring' 'by' 'exciting'
 'experience' 'fantastic' 'film' 'finish' 'from' 'fun' 'incredible' 'it'
 'lead' 'long' 'loved' 'movie' 'performance' 'plot' 'poor' 'start'
 'terrible' 'the' 'this' 'to' 'too' 'was' 'what']

=== N-gram Analysis ===


In [75]:
for ngram_range in [(1,1), (1,2), (2,2)]:
    vec = CountVectorizer(ngram_range=ngram_range)
    X = vec.fit_transform(reviews)
    print(f"\nN-gram range: {ngram_range}")
    print(f"Number of features: {len(vec.get_feature_names_out())}")
    print(f"Sample features: {vec.get_feature_names_out()[:10]}")


N-gram range: (1, 1)
Number of features: 31
Sample features: ['absolutely' 'acting' 'actor' 'an' 'and' 'boring' 'by' 'exciting'
 'experience' 'fantastic']

N-gram range: (1, 2)
Number of features: 61
Sample features: ['absolutely' 'absolutely loved' 'acting' 'actor' 'an' 'an incredible'
 'and' 'and exciting' 'and poor' 'and too']

N-gram range: (2, 2)
Number of features: 30
Sample features: ['absolutely loved' 'an incredible' 'and exciting' 'and poor' 'and too'
 'boring and' 'by the' 'exciting experience' 'experience from' 'film was']


In [76]:
def top_tfidf_terms(X, vectorizer, top_n=5):
    feature_array = np.array(vectorizer.get_feature_names_out())
    tfidf_sorting = np.argsort(X.toarray().sum(axis=0))[::-1]
    return feature_array[tfidf_sorting][:top_n]

print("\nTop TF-IDF Terms:", top_tfidf_terms(X_tfidf, tfidf_vec))


Top TF-IDF Terms: ['and' 'was' 'the' 'plot' 'poor']


### TF-IDF Feature Analysis

- **Top TF-IDF Terms**: Terms with high scores usually occur frequently in one review but are rare in others. This makes them strong discriminators.
- **Unigrams vs Bigrams**:
  - Unigrams ((1,1)): Highlighted individual words like "loved", "boring", "amazing".
  - Bigrams ((1,2)): Captured meaningful phrases like "not good", "absolutely loved", which improve feature richness.
- **Feature Count Growth**: As expected, bigrams and trigrams drastically increase the number of features.

Choosing the right ngram_range is a trade-off between expressiveness and model complexity.


In [77]:

# Exercise 3: Classification

X = tfidf_vec.fit_transform(reviews)
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

models = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression()
}

In [78]:
print("\n=== Classification Results ===")
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"\n{name} Classification Report:")
    print(classification_report(y_test, preds))


=== Classification Results ===

Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0


Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/cod

In [79]:
def evaluate_with_preprocessing(preprocess_func):
    processed = [" ".join(preprocess_func(review)) for review in reviews]
    X = tfidf_vec.fit_transform(processed)
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)
    model = LogisticRegression()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(classification_report(y_test, preds))

In [80]:
print("\n=== Preprocessing Impact ===")
print("\n-> No Preprocessing:")
evaluate_with_preprocessing(lambda x: x.split())



=== Preprocessing Impact ===

-> No Preprocessing:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/cod

In [81]:
print("\n-> NLTK Preprocessing:")
evaluate_with_preprocessing(nltk_preprocess)


-> NLTK Preprocessing:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/cod

In [82]:
print("\n-> spaCy Preprocessing:")
evaluate_with_preprocessing(spacy_preprocess)


-> spaCy Preprocessing:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/cod

In [83]:
print("\n=== Feature Selection (SelectKBest) ===")
X = tfidf_vec.fit_transform(reviews)
y = np.array(labels)


=== Feature Selection (SelectKBest) ===


In [84]:
for k in [5, 10, 15]:
    selector = SelectKBest(score_func=chi2, k=min(k, X.shape[1]))
    X_new = selector.fit_transform(X, y)
    mask = selector.get_support()
    selected_features = tfidf_vec.get_feature_names_out()[mask]
    print(f"\nTop {k} features: {selected_features}")


Top 5 features: ['acting' 'plot' 'poor' 'terrible' 'too']

Top 10 features: ['acting' 'and' 'boring' 'film' 'long' 'plot' 'poor' 'terrible' 'this'
 'too']

Top 15 features: ['absolutely' 'acting' 'and' 'boring' 'fantastic' 'film' 'it' 'long'
 'loved' 'movie' 'plot' 'poor' 'terrible' 'this' 'too']


## Summary

- Preprocessing plays a crucial role in text classification, with spaCy offering more refined results than NLTK.
- Feature extraction using TF-IDF with appropriate n-gram ranges enhances context understanding.
- Logistic Regression consistently outperforms Naive Bayes for this dataset.
- Feature selection using SelectKBest can reduce dimensionality without much loss in accuracy.

 Overall, a well-preprocessed pipeline with TF-IDF, bigrams, and Logistic Regression gives the best classification performance.
